In [1]:
!apt update
!apt purge *nvidia* -y
!apt install nvidia-driver-530 -y

!pip install pyopencl[pocl]
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')
import pyopencl as cl
import pyopencl.array as cl_array
import numpy as np

Leyendo lista de paquetes... Hecho
E: No se pudo abrir el fichero de bloqueo «/var/lib/apt/lists/lock» - open (13: Permiso denegado)
E: No se pudo bloquear el directorio /var/lib/apt/lists/
W: Se produjo un problema al desligar el fichero /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permiso denegado)
W: Se produjo un problema al desligar el fichero /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permiso denegado)
E: No se pudo abrir el fichero de bloqueo «/var/lib/dpkg/lock-frontend» - open (13: Permiso denegado)
E: No se pudo obtener el bloqueo de la interfaz dpkg (/var/lib/dpkg/lock-frontend). ¿Es usted superusuario?
E: No se pudo abrir el fichero de bloqueo «/var/lib/dpkg/lock-frontend» - open (13: Permiso denegado)
E: No se pudo obtener el bloqueo de la interfaz dpkg (/var/lib/dpkg/lock-frontend). ¿Es usted superusuario?
Defaulting to user installation because normal site-packages is not writeable


In [2]:
!lscpu
!nvidia-smi

Arquitectura:                            x86_64
  modo(s) de operación de las CPUs:      32-bit, 64-bit
  Address sizes:                         36 bits physical, 48 bits virtual
  Orden de los bytes:                    Little Endian
CPU(s):                                  2
  Lista de la(s) CPU(s) en línea:        0,1
ID de fabricante:                        GenuineIntel
  Nombre del modelo:                     Intel(R) Core(TM)2 Duo CPU     P8700  @
                                          2.53GHz
    Familia de CPU:                      6
    Modelo:                              23
    Hilo(s) de procesamiento por núcleo: 1
    Núcleo(s) por «socket»:              2
    «Socket(s)»                          1
    Revisión:                            10
    CPU MHz máx.:                        2527,0000
    CPU MHz mín.:                        798,0000
    BogoMIPS:                            5041.87
    Indicadores:                         fpu vme de pse tsc msr pae mce cx8 apic
  

## Kernel

In [3]:
# Write Kernel Code
C_elem_KernelSource = """
__kernel void parallelSum(
    __global const float *a_g,
    __global const float *b_g,
    __global float *res_g)
{
  int gid = get_global_id(0);
  res_g[gid] = a_g[gid] + b_g[gid];
}
"""

In [4]:
# Vector lenght
N = 2048

# Discover and Select Platform
platform_list = cl.get_platforms()

# Discover and Select Device
devices = platform_list[0].get_devices(device_type=cl.device_type.GPU)

# Create Context
context = cl.Context(devices=devices)

# Create command Queue
queue = cl.CommandQueue(context)

# Create and initialize input vector (Numpy)
mf = cl.mem_flags
a_np = np.arange(N).astype(np.float32)
b_np = np.arange(N).astype(np.float32)

# Create output vector with the same size as the input one (Numpy)
res_np = np.empty_like(a_np)

# Create and initialize input buffers (OpenCL)
a_g = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
b_g = cl.Buffer(context, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_np)

# Create output buffer with the same size as the input one (OpenCL)
res_g = cl.Buffer(context, mf.WRITE_ONLY, a_np.nbytes)

# Create Program
program = cl.Program(context, C_elem_KernelSource).build()

# Create Kernel
kernel = program.parallelSum

# Set Kernel Arguments
kernel.set_scalar_arg_dtypes([np.int32, None, None, None])
kernel.set_args(N, a_g, b_g, res_g)




# Define Global and Local Range Dimension and size
globalrange = res_np.shape
localrange = None

# Excecute Kernel
ev = cl.enqueue_nd_range_kernel(queue, kernel, globalrange, localrange)

# Copy Result from Device to Host
cl.enqueue_copy(queue, res_np, res_g)

# Check on CPU with Numpy:
assert np.allclose(res_np, a_np + b_np)

LogicError: Context failed: INVALID_VALUE

In [6]:
print(res_np)

[0.000e+00 2.000e+00 4.000e+00 ... 4.090e+03 4.092e+03 4.094e+03]
